In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import os
import shutil
import pandas as pd
from matplotlib import pyplot as plt
from skimage.feature import local_binary_pattern

In [ ]:
# Paths (add your file paths)
DATASET_DIR = '/content/drive/MyDrive/CBIR_ENDSEM/preprocessed'
QUERY_DIR = '/content/drive/MyDrive/CBIR_ENDSEM/query'
RESULTS_FINAL = '/content/drive/MyDrive/CBIR_ENDSEM/results_Final'

os.makedirs(RESULTS_FINAL, exist_ok=True)

# Parameters
COLOR_BINS = (4,4,4)
LBP_RADIUS = 1
LBP_POINTS = 8
LBP_METHOD = 'uniform'
GRID_SIZE = (3,3)

# Feature weights
WEIGHT_COLOR = 0.4
WEIGHT_TEXTURE = 0.4
WEIGHT_SHAPE = 0.2

TOP_K = 5

In [ ]:
# Functions
def get_label_from_filename(filename):
    name = filename.lower()

    natural_labels = ["butterfly","desert","kangaroo","lotus","mango","peacock",
                      "rose","sheep","sunflower","tiger","vulture","waterfall",
                      "watermelon","zebra"]

    for label in natural_labels:
        if label in name:
            return label

    if "gl" in name: return "glioma"
    if "me" in name: return "meningioma"
    if "no" in name: return "no_tumor"

    if "pichwai" in name: return "pichwai"
    if "gond" in name: return "gond"
    if "cow" in name: return "cow"
    if "t3" in name: return "face"

    return "unknown"

def crop_black_border(image, tol=10):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = gray > tol
    if mask.any():
        coords = np.argwhere(mask)
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0) + 1
        return image[y0:y1, x0:x1]
    return image

def segment_object(image):
    image = crop_black_border(image)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = (hsv[:,:,1] > 30) & (hsv[:,:,2] > 30)
    mask = mask.astype(np.uint8) * 255
    contours,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x,y,w,h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        return image[y:y+h, x:x+w]
    return image

def extract_spatial_color(image, bins=COLOR_BINS, grid=GRID_SIZE):
    image = segment_object(image)
    h,w = image.shape[:2]
    gx,gy = grid
    features=[]
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    for i in range(gx):
        for j in range(gy):
            x0,x1 = int(i*h/gx), int((i+1)*h/gx)
            y0,y1 = int(j*w/gy), int((j+1)*w/gy)
            hsv_block = hsv[x0:x1, y0:y1]
            lab_block = lab[x0:x1, y0:y1]
            h1 = cv2.calcHist([hsv_block],[0,1,2],None,bins,[0,180,0,256,0,256])
            h1 = cv2.normalize(h1,h1).flatten()
            h2 = cv2.calcHist([lab_block],[0,1,2],None,bins,[0,256,0,256,0,256])
            h2 = cv2.normalize(h2,h2).flatten()
            features.extend(h1)
            features.extend(h2)
    features = np.array(features)
    return features/(np.linalg.norm(features)+1e-7)

def extract_block_lbp(image, radius=1, n_points=8, method='uniform', grid=GRID_SIZE):
    image = segment_object(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h,w = gray.shape
    gx,gy = grid
    features=[]
    for i in range(gx):
        for j in range(gy):
            x0,x1 = int(i*h/gx), int((i+1)*h/gx)
            y0,y1 = int(j*w/gy), int((j+1)*w/gy)
            block = gray[x0:x1, y0:y1]
            lbp = local_binary_pattern(block,n_points,radius,method)
            hist,_ = np.histogram(lbp.ravel(), bins=np.arange(0,n_points+3), range=(0,n_points+2))
            hist = hist.astype("float")/(hist.sum()+1e-7)
            features.extend(hist)
    features = np.array(features)
    return features/(np.linalg.norm(features)+1e-7)

def extract_hu_moments(image):
    image = segment_object(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hu = cv2.HuMoments(cv2.moments(gray)).flatten()
    hu = -np.sign(hu)*np.log10(np.abs(hu)+1e-7)
    return hu/(np.linalg.norm(hu)+1e-7)

def cosine_similarity(a,b):
    return np.dot(a,b) / ((np.linalg.norm(a)*np.linalg.norm(b))+1e-7)

def compute_AP(retrieved_labels, true_label, Total_Relevant):
    precision_sum = 0
    relevant_found = 0

    for k in range(1, len(retrieved_labels) + 1):
        if retrieved_labels[k-1] == true_label:
            relevant_found += 1
            P_at_k = relevant_found / k
            precision_sum += P_at_k

    if Total_Relevant == 0:
        return 0

    return precision_sum / Total_Relevant


In [ ]:
# Load dataset
dataset_features=[]
dataset_images=[]
dataset_labels=[]

for domain in os.listdir(DATASET_DIR):
    folder = os.path.join(DATASET_DIR, domain)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None: continue

        color = extract_spatial_color(img)
        texture = extract_block_lbp(img)
        shape = extract_hu_moments(img)

        feat = np.hstack([
            WEIGHT_COLOR*color,
            WEIGHT_TEXTURE*texture,
            WEIGHT_SHAPE*shape
        ])

        dataset_features.append(feat)
        dataset_images.append(img_path)
        dataset_labels.append(get_label_from_filename(img_name))



dataset_features = np.array(dataset_features)
rows = []
AP_list = []

In [ ]:
# Process all images in Query folder
for query_file in os.listdir(QUERY_DIR):

    qpath = os.path.join(QUERY_DIR, query_file)
    qimg = cv2.imread(qpath)
    if qimg is None: continue

    qlabel = get_label_from_filename(query_file)

    # extract features for query
    qc = extract_spatial_color(qimg)
    qt = extract_block_lbp(qimg)
    qs = extract_hu_moments(qimg)

    qfeat = np.hstack([
        WEIGHT_COLOR*qc,
        WEIGHT_TEXTURE*qt,
        WEIGHT_SHAPE*qs
    ])

    # Similarities
    sims = np.array([cosine_similarity(qfeat, f) for f in dataset_features])

    # Retrieve top-K
    idxs = np.argsort(sims)[::-1][:TOP_K]
    retrieved_labels = [dataset_labels[i] for i in idxs]

    # Evaluation metrics
    TP = sum([1 for lab in retrieved_labels if lab == qlabel])
    FP = TOP_K - TP
    Total_Relevant = sum([1 for lab in dataset_labels if lab == qlabel])
    FN = max(Total_Relevant - TP, 0)
    TN = len(dataset_labels) - (TP + FP + FN)

    # Evaluation metrics
    Precision = TP / TOP_K
    Recall = TP / (Total_Relevant + 1e-7)
    F1 = 2 * Precision * Recall / (Precision + Recall + 1e-7)
    Accuracy = TP / TOP_K

    AP = compute_AP(retrieved_labels, qlabel, Total_Relevant)
    AP_list.append(AP)

    # Save results
    qfolder = os.path.join(RESULTS_FINAL, query_file.split('.')[0])
    os.makedirs(qfolder, exist_ok=True)
    shutil.copy(qpath, os.path.join(qfolder, f"query_{query_file}"))

    for i, idx in enumerate(idxs):
        shutil.copy(dataset_images[idx],
            os.path.join(qfolder, f"match_{i+1}_{os.path.basename(dataset_images[idx])}")
        )

    rows.append([
        query_file, qlabel, TOP_K,
        TP, FP, FN, TN,
        round(Precision,3), round(Recall,3),
        round(F1,3), round(Accuracy,3),
        round(AP,3), Total_Relevant
    ])

In [ ]:
# Save Evaluation mxetrics
df = pd.DataFrame(rows, columns=[
    "Query","Label","TopK","TP","FP","FN","TN",
    "Precision","Recall","F1","Accuracy",
    "AP","Total_Relevant"
])

df.to_excel(os.path.join(RESULTS_FINAL, "evaluation_metrics.xlsx"), index=False)

print("Evaluation complete")
print("Excel file saved at:", os.path.join(RESULTS_FINAL, "evaluation_metrics.xlsx"))
print("Images saved inside:", RESULTS_FINAL)


Evaluation complete
Excel file saved at: /content/drive/MyDrive/CBIR_ENDSEM/results_Final/evaluation_metrics.xlsx
Images saved inside: /content/drive/MyDrive/CBIR_ENDSEM/results_Final
